In [2]:
import pandas as pd
import numpy as np
import os
import csv
import json
from pathlib import Path
# os.environ['cuda_visible_device'] = '0'


In [3]:
# patient_id_file = Path('../all_patients_id.txt')
# all_patient_id = []
# with open(patient_id_file, 'r') as f:
#     all_patient_id = f.read().splitlines()
#     all_patient_id = np.unique(all_patient_id)
feature_dir = '../features/'
dirs = [x for x in Path(feature_dir).iterdir() if x.is_file()]
all_patient_id = []
for d in dirs:
    patient = str(d).split('/')[-1].split('.')[0]
    all_patient_id.append(patient)
all_patient_id

['TCGA-BL-A3JM-01',
 'TCGA-CF-A47V-01',
 'TCGA-4Z-AA89-01',
 'TCGA-XF-A9SK-01',
 'TCGA-XF-AAMW-01',
 'TCGA-GU-A42R-01',
 'TCGA-UY-A8OC-01',
 'TCGA-GV-A3QI-01',
 'TCGA-XF-AAMF-01',
 'TCGA-GV-A3JZ-01',
 'TCGA-E7-A3Y1-01',
 'TCGA-K4-A54R-01',
 'TCGA-CF-A9FM-01',
 'TCGA-E7-A4IJ-01',
 'TCGA-2F-A9KR-01',
 'TCGA-E7-A7DV-01',
 'TCGA-YC-A8S6-01',
 'TCGA-XF-AAN8-01',
 'TCGA-K4-A4AB-01',
 'TCGA-H4-A2HQ-01',
 'TCGA-YC-A89H-01',
 'TCGA-CU-A3YL-01',
 'TCGA-5N-A9KI-01',
 'TCGA-XF-A8HH-01',
 'TCGA-K4-A5RI-01',
 'TCGA-UY-A9PF-01',
 'TCGA-E7-A8O7-01',
 'TCGA-ZF-AA5H-01',
 'TCGA-HQ-A5ND-01',
 'TCGA-FD-A62O-01',
 'TCGA-ZF-AA4N-01',
 'TCGA-GU-AATP-01',
 'TCGA-BT-A42B-01',
 'TCGA-FD-A3B7-01',
 'TCGA-FD-A3B3-01',
 'TCGA-XF-AAMQ-01',
 'TCGA-XF-AAMH-01',
 'TCGA-DK-A1AB-01',
 'TCGA-GC-A6I1-01',
 'TCGA-FD-A5BS-01',
 'TCGA-FD-A43P-01',
 'TCGA-K4-AAQO-01',
 'TCGA-FD-A6TK-01',
 'TCGA-GV-A3JV-01',
 'TCGA-DK-A1AD-01',
 'TCGA-C4-A0F6-01',
 'TCGA-ZF-A9RG-01',
 'TCGA-FD-A3SS-01',
 'TCGA-ZF-A9R7-01',
 'TCGA-CU-A3KJ-01',


In [11]:
label_file = '../genomic_data/mRNA expression z-scores relative to normal samples (log RNA Seq V2 RSEM).txt'
label_pd = pd.read_csv(label_file, sep="\t", header=1)
gene_of_interest = label_pd.SAMPLE_ID.values
output = {}
patients_with_label = label_pd.columns[1:].values

with_label = []

if os.path.exists('../No_label_patient.txt'):
    with open('../No_label_patient.txt', 'r') as fread:
        without_label = fread.read().splitlines()
else:
    without_label = []

for patient in all_patient_id:
    if patient not in patients_with_label:
        print(f"This patient {patient} is not in the label file")
        without_label.append(patient)
        continue
    with_label.append(patient)
    if patient not in output:
        output[patient] = {}
    for gene in gene_of_interest:
        _row = label_pd[label_pd['SAMPLE_ID'] == gene]
        try:
            label = _row.loc[:, patient]
        except:
            # print(f"Couldn't find a label for gene {gene} for patient {patient}")
            continue
        if not np.isnan(label.values[0]):
            output[patient][gene] = label.values[0]
print(len(output.keys()))

This patient TCGA-XF-AAMF-01 is not in the label file
This patient TCGA-BT-A42B-01 is not in the label file
This patient TCGA-ZF-A9RG-01 is not in the label file
82


In [12]:
output

{'TCGA-BL-A3JM-01': {'MMP1': -0.3987,
  'MMP2': -3.0891,
  'MMP3': -1.6555,
  'MMP7': -0.1861,
  'MMP9': 3.0197,
  'MMP10': -1.1257,
  'MMP11': 2.1835,
  'MMP12': 0.4707,
  'MMP13': 1.4396,
  'MMP14': -0.3737,
  'MMP15': -2.226,
  'MMP16': -0.2072,
  'MMP17': -1.7944,
  'MMP19': -1.7642,
  'MMP21': -0.3115,
  'MMP23B': -2.5916,
  'MMP24': -0.4872,
  'MMP25': -1.3888,
  'MMP27': -2.3581,
  'MMP28': -2.4321,
  'ITGB3': -0.5332,
  'ITGAV': -0.9044,
  'PTK2': 0.5138,
  'CDH1': 0.507,
  'SPARC': -0.1888,
  'WFDC2': -1.0958},
 'TCGA-CF-A47V-01': {'MMP1': -1.0589,
  'MMP2': -1.4097,
  'MMP3': -1.673,
  'MMP7': 1.2222,
  'MMP9': -0.6535,
  'MMP10': -0.9503,
  'MMP11': 0.959,
  'MMP12': -0.8034,
  'MMP13': 0.2727,
  'MMP14': 1.1741,
  'MMP15': 1.6218,
  'MMP16': -1.2825,
  'MMP17': -4.4373,
  'MMP19': 2.2676,
  'MMP21': -0.6136,
  'MMP23B': -2.4134,
  'MMP24': 2.4829,
  'MMP25': -0.5394,
  'MMP27': -2.3581,
  'MMP28': 0.2946,
  'ITGB3': -0.8136,
  'ITGAV': -0.2098,
  'PTK2': 1.144,
  'CDH1': 0.

In [13]:
# header = ['id', 'label']
# fname = f'../data/labels/label_{gene_name}.csv'
# with open(fname, 'w', newline='') as myfile:
#     writer = csv.DictWriter(myfile, fieldnames=header)
#     writer.writeheader()
#     writer.writerows(values)

with open('../genomic_data/label_summary.json', 'w') as fwrite:
    json.dump(output, fwrite, indent=4)
    
without_label = np.unique(without_label)
with open('../No_label_patient.txt', 'w') as fwrite:
    for p in without_label:
        fwrite.write(p)
        fwrite.write('\n')
        
with open('../valid_patients.txt', 'w') as fwrite:
    for p in with_label:
        fwrite.write(p)
        fwrite.write('\n')